# Identificación condicionando en observables

## Propensity Score: NSW Training program

El siguiente ejercicio está basado en el capítulo 5 de [Causal Inference: the mixtape](https://mixtape.scunning.com/) de Scott Cunningham.

Usaremos datos de un experimento en el que se le dió un entrenamiento para el trabajo a individuos con dificultades para insertarse en el mercado laboral. La variable de resultado son los ingresos posteriores al entrenamiento. Como grupo de control se usará una muestra no experimental de la encuesta de población, CPS ¿Por qué esta muestra de control introduce sesgo de selección?

### Importar librerias y cargar los datos

In [ ]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from itertools import combinations 
import plotnine as p9

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
def read_data(file): 
    return pd.read_stata("https://github.com/scunning1975/mixtape/raw/master/" + file)

nsw_dw = read_data('nsw_mixtape.dta')
nsw_dw_cpscontrol = read_data('cps_mixtape.dta')

nsw_dw_cpscontrol = pd.concat((nsw_dw_cpscontrol, nsw_dw))

In [ ]:
print(nsw_dw_cpscontrol.shape)
nsw_dw_cpscontrol.head()

In [ ]:
# Frequency table for data_id
print("Frequency table for data_id:")
print("="*40)
freq_table = nsw_dw_cpscontrol['data_id'].value_counts().sort_index()
print(freq_table)

print("Frequency table for treat:")
print("="*40)
freq_table_treat = nsw_dw_cpscontrol['treat'].value_counts().sort_index()
print(freq_table_treat)


In [ ]:
# Two-way table (cross-tabulation) for treat and data_id
print("Two-way table: treat x data_id")
print("="*50)

# Create cross-tabulation
crosstab = pd.crosstab(nsw_dw_cpscontrol['treat'], 
                       nsw_dw_cpscontrol['data_id'], 
                       margins=True, 
                       margins_name="Total")
print(crosstab)



In [ ]:
# Creación de variables
nsw_dw_cpscontrol['u74'], nsw_dw_cpscontrol['u75'] = 0, 0
nsw_dw_cpscontrol.loc[nsw_dw_cpscontrol.re74==0, 'u74'] = 1
nsw_dw_cpscontrol.loc[nsw_dw_cpscontrol.re75==0, 'u75'] = 1

In [ ]:
# Filter out observations where data_id is "Dehejia-Wahba Sample" and treat is 0
# Keep: 1) All CPS observations, 2) Treated units from Dehejia-Wahba Sample
print("Before filtering:")
print(f"Total observations: {len(nsw_dw_cpscontrol)}")
print(nsw_dw_cpscontrol.groupby(['data_id', 'treat']).size())

# Create filter condition
# Keep if: NOT (data_id == "Dehejia-Wahba Sample" AND treat == 0)
filter_condition = ~((nsw_dw_cpscontrol['data_id'] == 'Dehejia-Wahba Sample') & 
                     (nsw_dw_cpscontrol['treat'] == 0))

# Apply filter
nsw_dw_cpscontrol = nsw_dw_cpscontrol[filter_condition].copy()

print("\nAfter filtering:")
print(f"Total observations: {len(nsw_dw_cpscontrol)}")
print(nsw_dw_cpscontrol.groupby(['data_id', 'treat']).size())

### Estimación del propensity score

Antes de realizar el procedimiento, calculemos el ATE con los datos del experimento. Después analizamos bajo que circunstancias podemos recuperarlo con datos de control sin asignación aleatoria

In [ ]:
mean1 = nsw_dw[nsw_dw.treat==1].re78.mean()
mean0 = nsw_dw[nsw_dw.treat==0].re78.mean()
ate = np.unique(mean1 - mean0)[0]
print("The experimental ATE estimate is {:.2f}".format(ate))

La estimación del propensity score $p(X)$ se realiza a través de un modelo logit

In [ ]:
logit_formula = 'treat ~ age + I(age**2) + I(age**3) + educ + I(educ**2) +' \
'marr + nodegree + black + hisp + re74 + re75 + u74 + u75 + educ*re74'
logit_nsw = smf.glm(formula=logit_formula, 
                    family=sm.families.Binomial(),
                   data=nsw_dw_cpscontrol).fit()
                  
nsw_dw_cpscontrol['pscore'] = logit_nsw.predict(nsw_dw_cpscontrol)

print(nsw_dw_cpscontrol.groupby('treat')['pscore'].mean())

p9.ggplot(nsw_dw_cpscontrol, p9.aes(x='pscore')) + p9.geom_histogram(bins=50) +\
p9.facet_wrap("treat", scales='free')

Note que la media de $p(X)$ para el grupo de tratamiento es 0.42 y para el de control es de 0.007. El supuesto de full support (common support, overlap) siginifica que debemos tener para cada probabilidad unidades en el grupo de tratamiento y control. Note que para los no tratados el $p(X)$ está concentrado a la izquierda, luego no se cumple el supuesto de soporte común. La diferencia en la dsitribución entre tratados y control se debe a que los tratados tienen unas características específicas que definen una población particular: jóvenes, solteros, bajo nivel educativo y pertenecientes a una minoría étnica. En la muestra que define el grupo de control este tipo de cartacerísticas es rara. 

### Estimación

Para estimar usando el propensity score lo que hacemos es ponderar por el inverso de la probabilidad de inclusión. El resultado de Horvitz-Thompson muestra que bajo CIA la diferencia de promedios ponderados corresponde al ATE

In [ ]:

nsw_dw_cpscontrol['y1'] = nsw_dw_cpscontrol.treat * nsw_dw_cpscontrol.re78 / nsw_dw_cpscontrol.pscore
nsw_dw_cpscontrol['y0'] = (1 - nsw_dw_cpscontrol.treat) * nsw_dw_cpscontrol.re78 / (1 - nsw_dw_cpscontrol.pscore)
nsw_dw_cpscontrol['ht'] = nsw_dw_cpscontrol['y1'] - nsw_dw_cpscontrol['y0']

te_1 = nsw_dw_cpscontrol.ht.mean()

print("Treatment Effect (non-normalized, all data): {:.2f}".format(te_1))

El resultado es muy lejano al valor obtenido con los datos de la aleatorización ¿Por qué la podenración que estamos usando arroja resultados tan distantes del correcto? Piense en las probabilidades del grupo de control y lo que implica para la ponderación dado que no tenemos soporte común satisfactorio 


El histograma del propensity score revela que tenemos muchas observaciones con valores cercanos a cero. Una estrategia es recortar (trimming) los datos y mantener únicamente las observaciones cuyo propensity score está en el intervalo $[0.1,0.9]$

In [ ]:
nsw_dw_trimmed = nsw_dw_cpscontrol.drop(['d1', 'd0', 'y1', 'y0'], axis=1)
nsw_dw_trimmed = nsw_dw_trimmed[nsw_dw_trimmed.pscore.between(.1, .9)]

nsw_dw_trimmed['y1'] = nsw_dw_trimmed.treat * nsw_dw_trimmed.re78 / nsw_dw_trimmed.pscore
nsw_dw_trimmed['y0'] = (1 - nsw_dw_trimmed.treat) * nsw_dw_trimmed.re78 / (1 - nsw_dw_trimmed.pscore)
nsw_dw_trimmed['ht'] = nsw_dw_trimmed['y1'] - nsw_dw_trimmed['y0']

te_3 = nsw_dw_trimmed.ht.mean()

print("Treatment Effect (non-normalized, trimmed data): {:.2f}".format(te_3))


Como se observa, el ATE estimado usando la ponderación de Horvitz-Thompson con todos los datos es muy diferente al valor experimental.

### Para comparar usaremos la estimación por MCO

In [ ]:
model_ols_nocont='re78~treat'

nsw_olsnocont = smf.ols(formula=model_ols_nocont, data=nsw_dw_cpscontrol).fit()
treat_coeff = nsw_olsnocont.params['treat']
treat_se = nsw_olsnocont.bse['treat']
print("OLS Results for Treatment Variable:")
print("="*40)
print(f"Coefficient (treat): {treat_coeff:.2f}")
print(f"Standard Error:     {treat_se:.2f}")

In [ ]:
model_ols='re78~treat+age + I(age**2) + I(age**3) + educ + I(educ**2) +' \
'marr + nodegree + black + hisp + re74 + re75 + u74 + u75 + educ*re74'

nsw_ols = smf.ols(formula=model_ols, data=nsw_dw_cpscontrol).fit()
treat_coeff = nsw_ols.params['treat']
treat_se = nsw_ols.bse['treat']
print("OLS Results for Treatment Variable:")
print("="*40)
print(f"Coefficient (treat): {treat_coeff:.2f}")
print(f"Standard Error:     {treat_se:.2f}")

Si el control por $X$ o usando $p(X)$ garantiza, dado el conjunto correcto de controles, en presencia de soporte común, permite obtener el ATE ¿Por qué propensity score y la regresión arrojan resultados similares pero diferentes? ¿Qué ventaja tendría la regresión sobre el el propensity (matching) y que ventaja este último sobre la regresión?